In [1]:
## Data Preparation
# List of functions for loading German Traffic Sign Data Set for (Training and Testing)
def IntialiseEnv():
    nb_dir = os.path.split(os.getcwd())[0]
    if nb_dir not in sys.path:
        sys.path.append(nb_dir)
    invalid_path='/opt/ros/kinetic/lib/python2.7/dist-packages'
    if invalid_path in sys.path:
        sys.path.remove(invalid_path)

In [2]:
## Import libraries need to be imported, Dont forget to update requirements.txt!
import os
import sys
IntialiseEnv()
import glob
import numpy as np
import cv2
import warnings
import pandas as pd 
import matplotlib.pyplot as plt
import import_ipynb
from Feature_Extraction.Segmentation import FeatureExtraction

importing Jupyter notebook from /home/rana/Documents/Deep_Learning/Project/Traffic_Sign_Recognition_Detection/Source_Code/Feature_Extraction/Segmentation.ipynb


In [3]:
# Function to get category directory and corresponding data frame
def getDirAndDataFrame(_dir):
    train_category_dir= os.path.join(training_dataset_dir,_dir)
    train_csv_path= getCsvPath(train_category_dir)
    train_Data_frame = pd.read_csv(train_csv_path, delimiter=';')
    return train_category_dir, train_Data_frame

In [4]:
## Function to get csv path
def getCsvPath(train_path_dir):
     return os.path.join(train_path_dir,list(filter(lambda x: '.csv' in x, os.listdir(train_path_dir)))[0])

In [5]:
## Read image in path
def readImage(path):
    return cv2.imread(path)

In [6]:
def loadSampleImg(path):
    img=readImage(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,(80,80))
    return img

In [ ]:
## Training DataSet Directory
training_dataset_dir = '../DataSet/Training_DataSet/Final_Training/Images'

def LoadTrainDataSet():
    train_image_array=[]
    train_labels_array=[]

    for _dir in os.listdir(training_dataset_dir):

        # Get Directory for train with certain class ID and its associated dataframe
        train_category_dir, train_data_frame= getDirAndDataFrame(_dir)
        # Get ClassID for entire category
        dir_img_label=train_data_frame.iloc[0]['ClassId']

        for img_path in glob.glob(os.path.join(train_category_dir, '*.ppm')):

            #Load image in category path directory + Extraction of features
            img = readImage(img_path)
            img_features = FeatureExtraction(img)
            train_image_array.append(img_features)

            #Populate associated labels
            train_labels_array.append(dir_img_label)

    return train_image_array, train_labels_array
        